![nsdf](https://www.sci.utah.edu/~pascucci/public/NSDF-large.png)  
[National Science Data Fabric](https://nationalsciencedatafabric.org/) 

# Converting numpy data 


In [ ]:
import os ,sys, time, logging,shutil
from datetime import datetime
import numpy as np

# sys.path.append(r"C:\projects\OpenVisus\build\RelWithDebInfo")
# sys.path.append(r"C:\projects\openvisuspy\src")

# sys.path.append(r"C:\projects\OpenVisus\build\RelWithDebInfo")
import OpenVisus as ov
os.environ["VISUS_DISABLE_WRITE_LOCK"]="1"
logger= logging.getLogger("OpenVisus")

# uncomment for debugging
# ov.SetupLogger(logger, output_stdout=True) 

# Download a numpy file to convert

NOTE: **it's 6GB file** and will take a lot to download

In [ ]:
numpy_filename="/mnt/data/chess/assets/recon_combined_1_2_3_fullres.npy"

from openvisuspy.utils import DownloadObject
DownloadObject("s3://utah/assets/recon_combined_1_2_3_fullres.npy",numpy_filename)

# Load the data in memory

In [ ]:
t1 = time.time()
data=np.load(numpy_filename)
shape,dtype=data.shape,data.dtype
D,H,W=data.shape
m,M=np.min(data),np.max(data)
print(f"np.load done in {time.time()-t1} seconds \ndtype={data.dtype} \nshape={data.shape} \nc_size={W*H*D*4:,} \nm={m} M={M}")

# Show a slice along Z

In [ ]:
import matplotlib.pyplot as plt
from scipy import ndimage

def ShowData(data,cmap="viridis"):
	print("ShowImage",data.dtype,data.shape,np.min(data),np.max(data))
	plt.imshow(data, cmap=cmap)
	plt.colorbar()
	plt.show()

Z=200
slice=data[Z,:,:]
ShowData(slice,cmap="viridis") 

# Create the IDX file

In [ ]:
idx_filename="./remove-me/example-numpy/visus.idx"

# ************ dangerous (make sure you are in a tmp directory) ************ 
import os,sys,shutil
assert("remove-me" in idx_filename)
shutil.rmtree(os.path.dirname(idx_filename), ignore_errors=True)

fields=[ov.Field("data",str(data.dtype),"row_major")]
db=ov.CreateIdx(
	url=idx_filename,
	dims=[W,H,D],
	fields=fields,
	compression="raw")

print(db.getDatasetBody().toString())

# Write data to OpenVisus

In [ ]:
t1 = time.time()
db.write(data)
print(f"db.write done in {time.time() - t1} seconds")

# (OPTIONAL) Compress dataset using zip

Here the compression is disabled, but better to compress data in production

In [ ]:
do_compress=False
if do_compress:
    t1 = time.time()
    db.compressDataset(["zip"])
    print(f"db.compressDataset done in {time.time()-t1} seconds")

# Show OpenVisus data

In [ ]:
slice=db.read(x=[0,W],y=[0,H],z=[Z,Z+1], num_refinements=1)[0,:,:]
ShowData(slice)

# Show coarse to fine

In [ ]:
import os,sys
num_refinements=3
for slice in db.read(x=[0,W],y=[0,H],z=[Z,Z+1], num_refinements=num_refinements):
	ShowData(slice[0,:,:])